In [18]:
import os 
os.chdir("/endosome/work/PCDC/s202005/Research/TransferLearning/Shelburne/Codes")
from general.commonfuns import *
from general.imports import *
pd.set_option('mode.chained_assignment',None)

# Step 1:Get Pred

In [2]:
ShelInfo = pd.read_csv("/endosome/work/PCDC/s202005/Research/TransferLearning/\
Shelburne/Data/Train/TLPARP/shelburne_info.csv",index_col=[0])

In [6]:
RESDIC = '/endosome/work/PCDC/s202005/Research/TransferLearning/Shelburne/Results/TL/Single_1_Anti_2_Dense_1024_Ensemble/Epochs_100_Batch_128/'
CV_list = ["cv_0","cv_1","cv_2"]
for seed in range(100,121,1):
    seed_name = "Seed_"+str(seed)
    LRPT_list = sorted([ele for ele in os.listdir(os.path.join(RESDIC,seed_name)) if 'Epochs' in ele])
    for LRPT in LRPT_list:
        for cv in CV_list:
            tempdic = os.path.join(RESDIC,seed_name,LRPT,cv)
            if not os.path.exists(tempdic):
                print("{},{},{},doesn't exist".format(seed_name,LRPT,cv))
                continue
            train_test_ix = open_pickle(os.path.join(tempdic,"train_test_ix.pkl"))
            train_info = ShelInfo.iloc[train_test_ix["train_ix"],]
            test_info = ShelInfo.iloc[train_test_ix["test_ix"],].reset_index().rename(columns={"index":"sample_index"})

            if not os.path.exists(os.path.join(tempdic,"fit_train_val_ix.pkl")):
                continue
            train_val_ix = open_pickle(os.path.join(tempdic,"fit_train_val_ix.pkl"))
            fit_train_info = train_info.iloc[train_val_ix["train_ix"],].reset_index().rename(columns={"index":"sample_index"})
            fit_val_info = train_info.iloc[train_val_ix["val_ix"],].reset_index().rename(columns={"index":"sample_index"})

            ## get fit score and pred 
            if not os.path.exists(os.path.join(tempdic,"fit_score.pkl")):
                print("no fit score for {},{},{}".format(seed_name,LRPT,cv))
                continue
            fit_score = open_pickle(os.path.join(tempdic,"fit_score.pkl"))
            fit_pred = open_pickle(os.path.join(tempdic,"fit_pred.pkl"))

            fit_train_pred = pd.DataFrame(fit_pred['train_pred'],columns=["pred_prob"])
            fit_train = pd.concat([fit_train_info,fit_train_pred],axis=1)

            fit_val_pred = pd.DataFrame(fit_pred['val_pred'],columns=["pred_prob"])
            fit_val = pd.concat([fit_val_info,fit_val_pred],axis=1)

            fit_test_pred = pd.DataFrame(fit_pred['test_pred'],columns=["pred_prob"])
            fit_test = pd.concat([test_info,fit_test_pred],axis=1)

            ## get base PARP perform
            train_info.reset_index(inplace=True)
            train_info.rename(columns={"index":"sample_index"},inplace=True)
            basePARP_score = open_pickle(os.path.join(tempdic,"basePARP_score.pkl"))
            basePARP_pred = open_pickle(os.path.join(tempdic,"basePARP_pred.pkl"))

            base_train = pd.DataFrame(basePARP_pred["train_pred"],columns=["pred_prob"])
            base_train = pd.concat([train_info,base_train],axis=1)

            base_test = pd.DataFrame(basePARP_pred["test_pred"],columns=["pred_prob"])
            base_test = pd.concat([test_info,base_test],axis=1)

            ###### save
            save_dic = os.path.join(tempdic,"pred_summary")
            os.makedirs(save_dic,exist_ok=True)
            fit_train.to_csv(os.path.join(save_dic,"fit_train.csv"),index=False)
            fit_test.to_csv(os.path.join(save_dic,"fit_test.csv"),index=False)
            fit_val.to_csv(os.path.join(save_dic,"fit_val.csv"),index=False)
            base_train.to_csv(os.path.join(save_dic,"base_train.csv"),index=False)
            base_test.to_csv(os.path.join(save_dic,"base_test.csv"),index=False)

no fit score for Seed_118,DefaultWeights_Epochs_10_Batch_128,cv_0


# Step 2: Evaluate

In [14]:
def get_test_df(save_dic,threshold):
    base_df = pd.read_csv(os.path.join(save_dic,"base_test.csv"))
    base_df = base_df[["sample_index","pred_prob"]]
    base_df.rename(columns={'pred_prob':"pred_prob_PARP"},inplace=True)

    fit_df = pd.read_csv(os.path.join(save_dic,"fit_test.csv"))
    fit_df.rename(columns={'pred_prob':"pred_prob_LRPT"},inplace=True)
    
    pred_df = pd.merge(base_df,fit_df,on="sample_index")
    col_list = ["sample_index","bacteria","antibiotics",
                "antibiotics_index","phenotype","phenotypenum","pred_prob_PARP","pred_prob_LRPT"]
    pred_df = pred_df[col_list]
    pred_df["pred_label_PARP"] = pred_df.apply(lambda x: 1*(x["pred_prob_PARP"]>threshold),axis=1)
    pred_df["pred_label_LRPT"] = pred_df.apply(lambda x: 1*(x["pred_prob_LRPT"]>threshold),axis=1)
    return pred_df

In [15]:
def get_report(pred_df,y_true_col,y_pred_col,y_score,model,threshold):
    f1score = metrics.f1_score(y_true = pred_df[y_true_col],y_pred = pred_df[y_pred_col])
    precision_score = metrics.precision_score(y_true = pred_df[y_true_col],y_pred = pred_df[y_pred_col])
    accuracy_score = metrics.accuracy_score(y_true = pred_df[y_true_col],y_pred = pred_df[y_pred_col])
    balanced_accuracy_score = metrics.balanced_accuracy_score(y_true = pred_df[y_true_col],y_pred = pred_df[y_pred_col])
    recall_score = metrics.recall_score(y_true = pred_df[y_true_col],y_pred = pred_df[y_pred_col])
    roc_auc_score = metrics.roc_auc_score(y_true=pred_df[y_true_col], y_score=pred_df[y_score])
    report = pd.DataFrame([model,threshold,f1score,precision_score,accuracy_score,
                           balanced_accuracy_score,recall_score,roc_auc_score]).T
    report.columns=["model","threshold","f1score","precision_score","accuracy_score",
                    "balanced_accuracy_score","recall_score","roc_auc_score"]
    return report

In [19]:
RESDIC = '/endosome/work/PCDC/s202005/Research/TransferLearning/Shelburne/Results/TL/Single_1_Anti_2_Dense_1024_Ensemble/Epochs_100_Batch_128/'
CV_list = ["cv_0","cv_1","cv_2"]
for seed in range(100,121,1):
    if seed == 118:
        continue 
    seed_name = "Seed_"+str(seed)
    LRPT_list = sorted([ele for ele in os.listdir(os.path.join(RESDIC,seed_name)) if 'Epochs' in ele])
    for LRPT in LRPT_list:
        for cv in CV_list:
            tempdic = os.path.join(RESDIC,seed_name,LRPT,cv)
            save_dic = os.path.join(tempdic,"pred_summary")
            threshold=0.5
            try:
                pred_df = get_test_df(save_dic,threshold=threshold)
            except:
                #raise ValueError("don't exist")
                continue 

            y_true_col = "phenotypenum"
            y_pred_col = "pred_label_PARP"
            y_score = "pred_prob_PARP"
            model="PARP"
            PARP_df = get_report(pred_df=pred_df,y_true_col=y_true_col,
                              y_pred_col=y_pred_col,y_score=y_score,
                              model=model,threshold=threshold)

            y_pred_col = "pred_label_LRPT"
            y_score = "pred_prob_LRPT"
            model="LRPT"
            LRPT_df = get_report(pred_df=pred_df,y_true_col=y_true_col,
                              y_pred_col=y_pred_col,y_score=y_score,
                              model=model,threshold=threshold)
            Final_df = pd.concat([PARP_df,LRPT_df])
            Final_df["seed"] = seed_name
            Final_df["cv"] = cv
            Final_df["LRPT"] = LRPT
            Final_df.to_csv(os.path.join(save_dic,"metrics.csv"),index=False)

/home2/s202005/.conda/envs/Research/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/s202005/.conda/envs/Research/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/s202005/.conda/envs/Research/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/s202005/.conda/envs/Research/lib/python3.9/site-

# Step 3: Summary

In [20]:
RESDIC = '/endosome/work/PCDC/s202005/Research/TransferLearning/Shelburne/Results/TL/Single_1_Anti_2_Dense_1024_Ensemble/Epochs_100_Batch_128/'
CV_list = ["cv_0","cv_1","cv_2"]
summarylist=[]
for seed in range(100,121,1):
    seed_name = "Seed_"+str(seed)
    LRPT_list = sorted([ele for ele in os.listdir(os.path.join(RESDIC,seed_name)) if 'Epochs' in ele])
    for LRPT in LRPT_list:
        for cv in CV_list:
            tempdic = os.path.join(RESDIC,seed_name,LRPT,cv)
            try:
                metrics = pd.read_csv(os.path.join(tempdic,"pred_summary","metrics.csv"))
            except:
                continue
            summarylist.append(metrics)

In [21]:
Summary = pd.concat(summarylist)

In [26]:
Summary

,model,threshold,f1score,precision_score,accuracy_score,balanced_accuracy_score,recall_score,roc_auc_score,seed,cv,LRPT
0,PARP,0.5,0.559242,0.433824,0.772059,0.777718,0.786667,0.837518,Seed_100,cv_0,CopiedWeights_Epochs_100_Batch_128
1,LRPT,0.5,0.640625,0.773585,0.887255,0.755315,0.546667,0.847808,Seed_100,cv_0,CopiedWeights_Epochs_100_Batch_128
0,PARP,0.5,0.482051,0.373016,0.752451,0.724061,0.681159,0.790411,Seed_100,cv_1,CopiedWeights_Epochs_100_Batch_128
1,LRPT,0.5,0.477876,0.613636,0.855392,0.670578,0.391304,0.808901,Seed_100,cv_1,CopiedWeights_Epochs_100_Batch_128
0,PARP,0.5,0.545455,0.434211,0.730392,0.731447,0.733333,0.791964,Seed_100,cv_2,CopiedWeights_Epochs_100_Batch_128
...,...,...,...,...,...,...,...,...,...,...,...
1,LRPT,0.5,0.074074,0.500000,0.816176,0.515495,0.040000,0.451091,Seed_120,cv_0,DefaultWeights_Epochs_50_Batch_64
0,PARP,0.5,0.323353,0.203774,0.446078,0.580095,0.782609,0.709119,Seed_120,cv_1,DefaultWeights_Epochs_50_Batch_64
1,LRPT,0.5,0.028169,0.500000,0.830882,0.505771,0.014493,0.502757,Seed_120,cv_1,DefaultWeights_Epochs_50_Batch_64
0,PARP,0.5,0.408964,0.273408,0.482843,0.600524,0.811111,0.674196,Seed_120,cv_2,DefaultWeights_Epochs_50_Batch_64


In [25]:
Summary = pd.concat(summarylist)
save_dic = os.path.join(RESDIC,"summary")
os.makedirs(save_dic,exist_ok=True)
Summary.to_csv(os.path.join(save_dic,"test_summary_100_120.csv"))